In [49]:
class Vektor:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __repr__(self):
        x, y = self.koordinate()
        return ('{}(x = {}, y = {})'.format(self.__class__.__name__, x, y))
    
    def __add__(self, other):
        x1 = self.x
        x2 = other.x
        y1 = self.y
        y2 = other.y
        return(Vektor(x1 + x2, y1 + y2))
    
    def __sub__(self, other):
        x1 = self.x
        x2 = other.x
        y1 = self.y
        y2 = other.y
        return(Vektor(x1 - x2, y1 - y2))
    
    def __rmul__(self, konstanta):
        x1 = konstanta * self.x
        y1 = konstanta * self.y
        return(Vektor(x1, y1))
    
    def koordinate(self):
        return self.x, self.y
    
class Premica:
    def __init__(self, u1, u2): # u tocka; rabili bomo koordinate
        u1x, u1y = u1.koordinate()
        u2x, u2y = u2.koordinate()
        if u2x - u1x != 0:
            self.koef = (u2y -u1y) / (u2x - u1x)
            self.n = u1y - self.koef * u1x
        else:
            self.koef = "NaN"
            self.x = u1x
        
    def parametri(self):
        if self.koef != "NaN":
            return self.koef, self.n
        else:
            return self.x
    
    def __repr__(self):
        if self.koef != "NaN":
            k, n = self.parametri()
            return ('{}(y = {} * x + {})'.format(self.__class__.__name__, k, n))
        else:
            x = self.parametri()
            return ('{}(x = {})'.format(self.__class__.__name__, x))
    
class U:
    def __init__(self, a, b):
        self.a = a # a konstanta ... koliko "prvih" vektorjev uporabimo
        self.b = b
        
    def __repr__(self):
        a,b = self.podatki()
        return ('{}({}a, {}b)'.format(self.__class__.__name__, a, b))
    
    def podatki(self):
        return self.a, self.b
    
    def koordinate(self):
        vektor = self.a * A1 + self.b * A2
        koordinate = vektor.koordinate()
        return [koordinate[0], koordinate[1]]
    
    def pod(self, premica): 
        x, y = self.koordinate()
        if premica.koef != "NaN":
            if y <= premica.koef * x + premica.n:
                return True
            else:
                return False
        # sicer pa pod pomeni levo od navpicnice
        else:
            if x <= premica.x:
                return True
            else:
                return False

class V:
    def __init__(self, a, b):
        self.a = a # a konstanta ... koliko "prvih" vektorjev uporabimo
        self.b = b
        
    def __repr__(self):
        a,b = self.podatki()
        return ('{}({}a, {}b)'.format(self.__class__.__name__, a, b))
    
    def podatki(self):
        return self.a, self.b
    
    def koordinate(self):
        vektor = self.a * A1 + self.b * A2 + Vektor(1, 0)
        koordinate = vektor.koordinate()
        return [koordinate[0], koordinate[1]]
    
    def pod(self, premica): 
        x, y = self.koordinate()
        if premica.koef != "NaN":
            if y <= premica.koef * x + premica.n:
                return True
            else:
                return False
        # sicer pa pod pomeni levo od navpicnice
        else:
            if x <= premica.x:
                return True
            else:
                return False
            
        

A1 = Vektor(0, - sqrt(3))
A2 = Vektor(3 / 2, - sqrt(3) / 2)

def grid(k, l, m, n):
    # oglisca = ([0, 0], [k, l], [m, n], [k + m, l + n])
    # print U(0,0).koordinate(), U(k, l).koordinate(), U(m, n).koordinate(), U(k + m, l + n).koordinate()
    mina = min(0, k, m, k + m)
    maxa = max(0, k, m, k + m)
    minb = min(0, l, n, l + n)
    maxb = max(0, l, n, l + n)
    
    matrika = []
    for i in range(mina, maxa + 1):
        vrstica = []
        for j in range(minb, maxb + 1):
            vrstica += [U(i, j)]
            # print U(i,j), U(i, j).koordinate()
        matrika += [vrstica]
        
    return matrika
## način, da bi dobila desno in zgornjo stranico v paralelgramu???????? ########       

In [53]:
Uji = [U(-1,2), U(0,0), U(1,1), U(0,3)]
max(Uji, key = lambda y: y.koordinate()[1])

U(-1a, 2b)

In [45]:
p = Premica(U(0,0),U(-1,0))
U(-1,1).pod(p)

False

In [51]:
[max(Uji, key = lambda y: y.koordinate()[1])]

U(0a, 0b)